In [8]:
import numpy as np

In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
import json

In [11]:
from pandas.io.json import json_normalize

In [12]:
import requests

In [13]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

In [14]:
from sklearn.cluster import KMeans

In [15]:
import folium

In [16]:
CLIENT_ID = 'L5EVEQVLUNCAAZOSAKNUCRLIVURMRVNSWS0UPLJYGOA524KP' 
CLIENT_SECRET = 'TPA14IDRXS33ORW0R5S3BN0CQV03ESDWBSCJJ51DEIE0O304'
VERSION = '20180605' 
SAN_JOSE_LAT = "37.300876"
SAN_JOSE_LONG = "-121.871667"
radius = '5000'

def getUrl(categoryID):
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        SAN_JOSE_LAT,
        SAN_JOSE_LONG,
        radius,
        categoryID)
    return url


In [17]:
def getVenuesInSanJose(categoryID):
    results = requests.get(getUrl(categoryID)).json()
    venues = results['response']['venues']

    column_names = ['Name', 'Latitude', 'Longitude']
    venue_df = pd.DataFrame(columns=column_names)
    
    for venue in venues:
        a = venue["location"]["city"]

        if a == "San Jose":
            venue_df = venue_df.append({
                'Name' :venue['name'],
                'Latitude' : venue['location']['lat'],
                'Longitude' : venue['location']['lng']}, ignore_index=True)
    
    venue_df.head()
    
    return venue_df


In [18]:
playgrounds = getVenuesInSanJose("4bf58dd8d48988d1e7941735")
playgrounds

,Name,Latitude,Longitude
0,Rockin' Jump,37.307512,-121.865900
1,Children's Discovery Museum of San Jose,37.326595,-121.891782
2,Paul Moore Park,37.271115,-121.889444
3,Rotary Play Garden,37.341907,-121.904986
4,Happy Hollow Park & Zoo,37.325580,-121.861996
5,Ryland Park,37.343483,-121.898955
6,River Glen Park,37.301512,-121.888321
7,St. James Park,37.339691,-121.891584
8,Erikson Park,37.262456,-121.865983
9,Lemur Woods,37.325410,-121.861718


In [19]:
schools = getVenuesInSanJose('52e81612bcbc57f1066b7a45')
schools

,Name,Latitude,Longitude
0,Bright Horizons at Tamien,37.312176,-121.883140
1,Cryy Out Christian Fellowship,37.291694,-121.833595
2,Kidango Cesar Chavez,37.349150,-121.845253
3,A Place To Grow,37.267111,-121.919691
4,Kidango Chynoweth Center,37.256967,-121.861674
5,Kidango Meyer Center,37.335241,-121.826956
6,Action Day Primary Plus - Lincoln,37.297899,-121.894061
7,Kidango Dorsa Center,37.340844,-121.836705
8,Creative Minds,37.252422,-121.900813
9,Kidango Arbuckle,37.344234,-121.841385


In [22]:
map_SanJose = folium.Map(location=[SAN_JOSE_LAT, SAN_JOSE_LONG], zoom_start=10)

for lat, lng, name in zip(schools['Latitude'], schools['Longitude'], schools['Name']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SanJose)  
    
for lat, lng, name in zip(playgrounds['Latitude'], playgrounds['Longitude'], playgrounds['Name']):
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#00ff00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SanJose)  

map_SanJose

In [24]:
kclusters = 5
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=10)
kmeans.fit(schools[schools.columns[1:3]]) 
schools['Cluster Labels'] = kmeans.fit_predict(schools[schools.columns[1:3]])
centers = kmeans.cluster_centers_ 
labels = kmeans.predict(schools[schools.columns[1:3]]) 
school_cluster_count = schools.groupby('Cluster Labels').size()
print(school_cluster_count)
schools

Cluster Labels
0    2
1    4
2    6
3    3
4    3
dtype: int64


,Name,Latitude,Longitude,Cluster Labels
0,Bright Horizons at Tamien,37.312176,-121.883140,2
1,Cryy Out Christian Fellowship,37.291694,-121.833595,0
2,Kidango Cesar Chavez,37.349150,-121.845253,1
3,A Place To Grow,37.267111,-121.919691,4
4,Kidango Chynoweth Center,37.256967,-121.861674,3
5,Kidango Meyer Center,37.335241,-121.826956,1
6,Action Day Primary Plus - Lincoln,37.297899,-121.894061,2
7,Kidango Dorsa Center,37.340844,-121.836705,1
8,Creative Minds,37.252422,-121.900813,4
9,Kidango Arbuckle,37.344234,-121.841385,1


In [29]:
map_of_clusters_schools = folium.Map(location=[SAN_JOSE_LAT, SAN_JOSE_LONG], zoom_start=10)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, poi, cluster in zip(schools['Latitude'], schools['Longitude'], schools['Name'], schools['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_of_clusters_schools)


for center in centers:
    folium.CircleMarker(
        [center[0], center[1]],
        radius=5,
        popup='Center',
        color='black',
        fill=True,
        fill_color=r'black',
        fill_opacity=0.7).add_to(map_of_clusters_schools)

map_of_clusters_schools

In [26]:
kclusters = 5
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=10)
kmeans.fit(playgrounds[playgrounds.columns[1:3]]) 
playgrounds['Cluster Labels'] = kmeans.fit_predict(playgrounds[playgrounds.columns[1:3]])
playgroundCenters = kmeans.cluster_centers_  
centers = kmeans.cluster_centers_
labels = kmeans.predict(playgrounds[playgrounds.columns[1:3]])
playground_cluster_count = playgrounds.groupby('Cluster Labels').size()
print(playground_cluster_count)
playgrounds

Cluster Labels
0    6
1    8
2    8
3    3
4    5
dtype: int64


,Name,Latitude,Longitude,Cluster Labels
0,Rockin' Jump,37.307512,-121.865900,4
1,Children's Discovery Museum of San Jose,37.326595,-121.891782,2
2,Paul Moore Park,37.271115,-121.889444,0
3,Rotary Play Garden,37.341907,-121.904986,2
4,Happy Hollow Park & Zoo,37.325580,-121.861996,4
5,Ryland Park,37.343483,-121.898955,2
6,River Glen Park,37.301512,-121.888321,0
7,St. James Park,37.339691,-121.891584,2
8,Erikson Park,37.262456,-121.865983,1
9,Lemur Woods,37.325410,-121.861718,4


In [28]:
map_of_clusters_playgrounds = folium.Map(location=[SAN_JOSE_LAT, SAN_JOSE_LONG], zoom_start=10)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat, lon, poi, cluster in zip(playgrounds['Latitude'], playgrounds['Longitude'], playgrounds['Name'], playgrounds['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_of_clusters_playgrounds)


for center in playgroundCenters:
    folium.CircleMarker(
        [center[0], center[1]],
        radius=5,
        popup='Center',
        color='black',
        fill=True,
        fill_color=r'black',
        fill_opacity=0.7).add_to(map_of_clusters_playgrounds)

map_of_clusters_playgrounds

In [38]:
playgrounds_schools_ClusterCenterMap = folium.Map(location=[SAN_JOSE_LAT, SAN_JOSE_LONG], zoom_start=10)

for center in playgroundCenters:
    folium.CircleMarker(
        [center[0], center[1]],
        radius=5,
        popup='Center',
        color='green',
        fill=True,
        fill_color=r'green',
        fill_opacity=0.7).add_to(playgrounds_schools_ClusterCenterMap)


for center in centers:
    folium.CircleMarker(
        [center[0], center[1]],
        radius=5,
        popup='Center',
        color='blue',
        fill=True,
        fill_color=r'blue',
        fill_opacity=1).add_to(playgrounds_schools_ClusterCenterMap)

playgrounds_schools_ClusterCenterMap